In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# model to env
import shutil
shutil.unpack_archive('/content/drive/MyDrive/Colab Notebooks/merge268files_1/model_checkpoint_0.47_v15k-SL64-B400-6-14.tar.gz','model_checkpoint_0.47')

In [ ]:
# model to drive
import shutil
shutil.make_archive('/content/drive/MyDrive/Colab Notebooks/merge268files_1/model_checkpoint_0.36_v15k-SL256-B200-6-14', 'gztar', 'model_checkpoint_0.36')

In [ ]:
# bpe_tokenizer to env
import shutil
shutil.copy('/content/drive/MyDrive/Colab Notebooks/merge268files_1/bpe_tokenizer.json', 'bpe_tokenizer.json')

In [ ]:
# bpe_tokenizer to drive
import shutil
shutil.copy('bpe_tokenizer.json', '/content/drive/MyDrive/Colab Notebooks/merge268files_1/bpe_tokenizer_new15k.json')

In [ ]:
# tokens to env
import shutil
shutil.copy('/content/drive/MyDrive/Colab Notebooks/merge268files_1/tokens_15k.npy', 'tokens.npy')

In [ ]:
# tokens to drive
import shutil
shutil.copy('tokens.npy', '/content/drive/MyDrive/Colab Notebooks/merge268files_1/tokens_new15k.npy')

In [ ]:
# data_file to env
import shutil
shutil.copy('/content/drive/MyDrive/Colab Notebooks/merge268files_1/merge268files.txt', 'merge268files.txt')

In [ ]:
my_data_file = 'merge70files.txt'

In [ ]:
!pip install tokenizers

In [ ]:
# СОЗДАНИЕ СЛОВАРЯ

from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers
import time

# Создание модели BPE
bpe_model = models.BPE()

# Настройка токенизатора
tokenizer = Tokenizer(bpe_model)

# Добавление специальных токенов для пробелов и других символов
special_tokens = ["[END]", "[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "<space>"]
tokenizer.add_special_tokens(special_tokens)

# Использование пробельного пре-токенизатора
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=True)

# Настройка декодера для корректного восстановления текста
tokenizer.decoder = decoders.ByteLevel()

# Обучение токенизатора на корпусе данных
trainer = trainers.BpeTrainer(
    vocab_size=15000,
    special_tokens=special_tokens,
    show_progress=True
)
files = [my_data_file]
startt = time.time()
tokenizer.train(files, trainer)
print(time.time()-startt)

# Тестирование токенизатора
input_text = 'мое времяпровождение заключается в том чтобы создавать нейронные сети'
tokens = tokenizer.encode(input_text).ids
decoded_text = tokenizer.decode(tokens)

print("Original text:", input_text)
print("Tokens:", tokens)
print("Decoded text:", decoded_text)

# Сохранение токенизатора
tokenizer.save("bpe_tokenizer.json")

In [ ]:
# СОЗДАНИЕ СЛОВАРЯ new

from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers, processors
import time

# Создание модели BPE
bpe_model = models.BPE()

# Настройка токенизатора
tokenizer = Tokenizer(bpe_model)

# Добавление специальных токенов
special_tokens = ["[END]", "[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "<space>"]
tokenizer.add_special_tokens(special_tokens)

# Использование пробельного пре-токенизатора
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=True)

# Настройка пост-процессора для улучшенной сегментации
post_processor = processors.TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", tokenizer.token_to_id("[CLS]")),
        ("[SEP]", tokenizer.token_to_id("[SEP]")),
    ],
)
tokenizer.post_processor = post_processor

# Настройка декодера для корректного восстановления текста
tokenizer.decoder = decoders.ByteLevel()

# Обучение токенизатора на корпусе данных
trainer = trainers.BpeTrainer(
    vocab_size=15000,
    #min_frequency=2,
    special_tokens=special_tokens,
    #show_progress=True,
    #continuing_subword_prefix="##",
    #end_of_word_suffix="##",
    dropout=0.1
)
files = [my_data_file]  # Убедитесь, что путь к файлу данных верен
startt = time.time()
tokenizer.train(files, trainer)
print(f"Training time: {time.time()-startt:.2f} seconds")

# Тестирование токенизатора
input_text = 'мое времяпровождение заключается в том чтобы создавать нейронные сети'
encoded = tokenizer.encode(input_text)
tokens = encoded.ids
decoded_text = tokenizer.decode(tokens)

print("Original text:", input_text)
print("Tokens:", tokens)
print("Decoded text:", decoded_text)

# Сохранение токенизатора
tokenizer.save("bpe_tokenizer.json")

In [ ]:
my_model_path = 'models/model_checkpoint_0.20.keras'

In [ ]:
!pip install ipywidgets

In [ ]:
from tensorflow.keras.utils import get_custom_objects

get_custom_objects().update({'TransformerBlock': TransformerBlock})

In [ ]:
# ОБЩЕНИЕ С СЕТОЧКОЙ

import tensorflow as tf
import numpy as np
from tokenizers import Tokenizer
import ipywidgets as widgets
from IPython.display import display

# Загружаем модель и токенизатор
dir_path = ''
tokenizer = Tokenizer.from_file(dir_path + my_tokenizer_path)
model_path = dir_path + my_model_path
model = tf.keras.models.load_model(model_path)
#model.summary()

def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def tail_free_sampling(logits, tail_free_threshold=0.9, temperature=1.0):
    # Apply temperature scaling
    logits = logits / temperature

    # Convert logits into probabilities using softmax
    probs = softmax(logits)

    # Sort probabilities in descending order
    sorted_indices = np.argsort(probs)[::-1]
    sorted_probs = np.sort(probs)[::-1]

    # Calculate the first derivative
    first_derivative = np.diff(sorted_probs)

    # Calculate the second derivative
    second_derivative = np.diff(first_derivative)

    # Calculate the absolute values of the second derivative
    abs_second_derivative = np.abs(second_derivative)

    # Normalize the second derivative
    normalized_second_derivative = abs_second_derivative / np.sum(abs_second_derivative)

    # Find the smallest subset of second derivative magnitudes that surpass the threshold
    cumulative_sum = np.cumsum(normalized_second_derivative)
    cutoff_index = np.searchsorted(cumulative_sum, tail_free_threshold, side='right')

    # Keep only tokens up to the cutoff index
    if cutoff_index < len(sorted_probs):
        sorted_indices = sorted_indices[:cutoff_index + 1]
        sorted_probs = sorted_probs[:cutoff_index + 1]
    else:
        sorted_indices = sorted_indices[:1]
        sorted_probs = sorted_probs[:1]

    # Renormalize probabilities
    normalized_probs = sorted_probs / np.sum(sorted_probs)

    # Sample next token based on the normalized probabilities
    next_token = np.random.choice(sorted_indices, p=normalized_probs)

    return next_token

def generate_text(model, tokenizer, input_text, max_length=100, temperature=1.0, tail_free_threshold=0.9):
    tokens = tokenizer.encode(input_text).ids

    for _ in range(max_length):
        token_array = np.array(tokens)[None, :]  # Add batch dimension
        predictions = model.predict(token_array, verbose=0)
        next_token_logits = predictions[0, -1, :]

        next_token = tail_free_sampling(next_token_logits, tail_free_threshold=tail_free_threshold, temperature=temperature)
        tokens.append(next_token)

        # Отображаем ответ с добавлением новых токенов в той же строке
        #print(f"\rText: {tokenizer.decode(tokens)}", end='', flush=True)

        if next_token == tokenizer.token_to_id("[END]"):
            break
        if ')' in tokenizer.decode([next_token]):
            break

    return tokenizer.decode(tokens)

# Define generation parameters
temperature = 0.5
tail_free_threshold = 0.995
max_length = 50

# Взаимодействие с моделью
while False:
    #input_text = input("\nYou: ")
    input_text = my_input_text
    if input_text.lower() in ["exit", "quit"]:
        break
    response = generate_text(model, tokenizer, input_text, max_length=max_length, temperature=temperature, tail_free_threshold=tail_free_threshold)
    print(f"AI:{response}")
#for i in range(5,15+1, 2):
#    for j in range(5, 15+1, 2):
#        print(f"\ntemp={i*0.1} tfs={j*0.1}")
#        print(generate_text(model, tokenizer, "такой кайф", max_length=max_length, temperature=i * 0.1, tail_free_threshold=j * 0.1))

# Создаем виджеты
text_input = widgets.Textarea(
    value='Артем: ',
    placeholder='Введите текст...',
    description='AI:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='200px')  # Устанавливаем ширину и высоту
)

max_length_slider = widgets.IntSlider(
    value=max_length,
    min=1,
    max=201,
    step=5,
    description='Max length:',
    continuous_update=False
)
temperature_slider = widgets.FloatSlider(
    value=temperature,
    min=0.1,
    max=2.0,
    step=0.1,
    description='Temperature:',
    continuous_update=False
)

# Создаем логарифмический слайдер для tail_free_threshold
#log_scale = np.logspace(np.power(2,np.log10(0.05)), np.power(2, np.log10(1.0)), num=100)

tail_free_slider = widgets.FloatSlider(
    value=np.power(10, np.log10(tail_free_threshold)/np.log10(2)),
    min=0.05,
    max=1.0,
    step=0.05,
    description='Tail Free:',
    continuous_update=False
)

output_area = widgets.Output()

def on_button_click(b):
    with output_area:
        output_area.clear_output()
        prompt = text_input.value
        response = generate_text(model, tokenizer, prompt, max_length=max_length, temperature=temperature, tail_free_threshold=tail_free_threshold)
        text_input.value += response[len(text_input.value)+1:] + "\nАртем: "
        #print(f"\r{response}", end='', flush=True)
        print(temperature, tail_free_threshold, max_length)

button = widgets.Button(description="Сгенерировать текст")
button.on_click(on_button_click)

def update_params(change):
    global temperature
    global tail_free_threshold
    global max_length
    temperature = temperature_slider.value
    tail_free_threshold = np.power(1.25,np.log10(tail_free_slider.value))
    max_length = max_length_slider.value
    #print(f"\r{tail_free_threshold}", end='', flush=True)

temperature_slider.observe(update_params, names='value')
tail_free_slider.observe(update_params, names='value')

# Отображаем виджеты
display(text_input)
display(button)
display(temperature_slider)
display(tail_free_slider)
display(max_length_slider)
display(output_area)

In [ ]:
my_data_file2 = my_data_file

In [ ]:
my_tokenizer_path2 = 'bpe_tokenizer.json'

In [ ]:
!pip install tensorflow

In [ ]:
# ТОКЕНИЗАЦИЯ ДАТА ФАЙЛА

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
from tokenizers import Tokenizer
import os
import time

dir_path = ''
data_file = my_data_file2

print("Загрузка BPE-токенизатора")
tokenizer = Tokenizer.from_file(dir_path + my_tokenizer_path2)

# Получение словаря из токенизатора
vocab_size = tokenizer.get_vocab_size()
print(f"Вокабуляр содержит {vocab_size} слов")

# Функция для токенизации и записи токенов в файл
def tokenize_and_save(file_path, chunk_size=50000, token_file_path=dir_path + 'tokens.npy'):
    token_count = 0
    with open(file_path, 'r') as f:
        if os.path.exists(token_file_path):
            tokens = list(np.load(token_file_path))
            token_count = len(tokens)
        else:
            tokens = []

        while True:
            text_chunk = f.read(chunk_size)
            if not text_chunk:
                break
            tokens_chunk = tokenizer.encode(text_chunk).ids
            tokens.extend(tokens_chunk)
            token_count += len(tokens_chunk)
            print(f"\rТокенов: {token_count}", end='', flush=True)

        print()
        np.save(token_file_path, tokens)
        return token_count

if os.path.exists(dir_path + 'tokens.npy'):
    os.remove(dir_path + 'tokens.npy')

print("Токенизация и сохранение токенов в файл")
token_count = tokenize_and_save(dir_path + data_file)

In [ ]:
token_count = 51042083

In [ ]:
token_count = 30673990

In [ ]:
token_count = 22682187

In [ ]:
my_tokenizer_path = 'bpe_tokenizer.json'

In [ ]:
# ИМПОРТЫ БЕЗ ТОКЕНИЗАЦИИ ДАТА ФАЙЛА

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
from tokenizers import Tokenizer
import time
import os
import gc
from tensorflow.keras.callbacks import LearningRateScheduler

import keras

gc.enable()

print("Загрузка BPE-токенизатора")
tokenizer = Tokenizer.from_file(my_tokenizer_path)

# Получение словаря из токенизатора
vocab_size = tokenizer.get_vocab_size()
print(f"Вокабуляр содержит {vocab_size} слов")

In [ ]:
from transformers import GPTNeoXForCausalLM
model = GPTNeoXForCausalLM.from_pretrained(
  "EleutherAI/pythia-70m-deduped",
  revision="step143000",
  cache_dir="./pythia-70m-deduped/step143000",
)

# Save the entire model as a SavedModel.
!mkdir -p pythia
model.save_pretrained('pythia')

In [ ]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
dir = "pythia"

model = TFAutoModel.from_pretrained(
  "EleutherAI/pythia-70m-deduped",
  revision="step143000",
  cache_dir="./pythia-70m-deduped/step143000",)
model.save(dir)

In [ ]:
from transformers import AutoModelForCausalLM, TFAutoModelForCausalLM
#import tensorflow as tf

# Шаг 1: Загрузить модель PyTorch
torch_model = AutoModelForCausalLM.from_pretrained(
    "pythia",
    from_tf=False
)

In [ ]:
# Шаг 2: Сохраните весы модели в PyTorch формате (для безопасности)
torch_model.save_pretrained("./torch_model")

In [ ]:
from transformers import TFAutoModelForCausalLM
# Шаг 3: Загрузите модель TensorFlow
tf_model = TFAutoModelForCausalLM.from_pretrained(
    "./torch_model",
    from_pt=True
)

In [ ]:
# ЗАГРУЗКА ДАТА ФАЙЛА В RAM

dir_path = ''

# Загрузка токенов из файла
def load_tokens(token_file_path):
    tokens = np.load(token_file_path)
    return tokens

tokens = load_tokens(dir_path + 'tokens.npy')
token_count = len(tokens)
print(f"Загружено {token_count} токенов")

In [ ]:
create_nn = True
nn_file = 'model_checkpoint_0.41'
#nn_file = 'pythia'
model_scale = 4 # масштаб блока трансформера
num_transformer_blocks = 4 # количество блоков трансформера
use_lr_scheduler = False
initial_lr = 0.001 #1e-7 #0.001

sequence_length = 8 # размер сэмпла
batch_size = 4*50 #8*50 #int(150000/sequence_length) # размер батча

epochs = 1 #5
epochs_infit = 1 #5
# настройте это значение в зависимости от вашего датасета
steps_per_epoch = 50 #500 #15 #for lr_scheduler #500 #token_count // (batch_size * sequence_length)
save_freq= 50 #5000 #'epoch'

In [ ]:
# ОБУЧЕНИЕ СЕТИ

# Проверка и инициализация TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    if 'strategy' not in globals():
        tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    print('Running on TPU')
    print("All devices: ", tf.config.list_logical_devices('TPU'))
except ValueError:
    strategy = tf.distribute.get_strategy()
    print('Running on default strategy (CPU/GPU)')

class PrintLearningRate(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        lr = self.model.optimizer.learning_rate.numpy()
        print(f"\nLearning rate after epoch {epoch+1}: {lr:.4e}")

# Функция для разогрева learning rate
def lr_scheduler(epoch):
    warmup_epochs = 10  # Количество эпох для разогрева
    if epoch < warmup_epochs:
        return initial_lr * (epoch + 1) / warmup_epochs
    else:
        return initial_lr #* tf.math.exp(-0.1 * (epoch - warmup_epochs))

@keras.saving.register_keras_serializable()
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, trainable=True, dtype='float32', rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
tf.keras.utils.get_custom_objects().update({'TransformerBlock': TransformerBlock})

def create_model(create_nn, model_scale, num_transformer_blocks, nn_file, vocab_size):
    if create_nn:
        embed_dim = 16 * model_scale
        num_heads = 1 * model_scale
        ff_dim = 16 * model_scale

        inputs = tf.keras.Input(shape=(None,))
        embedding_layer = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        x = embedding_layer(inputs)
        for _ in range(num_transformer_blocks):
            transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
            x = transformer_block(x, training=True)
        outputs = layers.Dense(vocab_size, activation='softmax')(x)

        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    else:
        model = tf.keras.models.load_model(nn_file)
    return model

# RAM
# Функция для генерации батчей из массива токенов
def generate_batch_ram(tokens, batch_size, sequence_length):
    indices = np.random.randint(0, len(tokens) - sequence_length - 1, batch_size)
    X = np.zeros((batch_size, sequence_length), dtype=np.int64)
    Y = np.zeros((batch_size, sequence_length), dtype=np.int64)

    for i, idx in enumerate(indices):
        X[i] = tokens[idx:idx + sequence_length]
        Y[i] = tokens[idx + 1:idx + 1 + sequence_length]

    return X, Y

def generate_dataset_ram(tokens, batch_size, sequence_length):
    def generator():
        while True:
            yield generate_batch_ram(tokens, batch_size, sequence_length)
    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec(shape=(batch_size, sequence_length), dtype=tf.int64),
            tf.TensorSpec(shape=(batch_size, sequence_length), dtype=tf.int64)
        )
    )
    return dataset.repeat().prefetch(tf.data.AUTOTUNE)

distributed_dataset = strategy.distribute_datasets_from_function(
    lambda _: generate_dataset_ram(tokens, batch_size, sequence_length)
)

print(f"Создать модель - {create_nn}")
with strategy.scope():
    if 'model' in globals():
        del model
        gc.collect()
    model = create_model(create_nn, model_scale, num_transformer_blocks, dir_path + nn_file, vocab_size)

#model.summary()  # вывод информации о модели

# Настройка callback для сохранения модели
checkpoint_filepath = 'model_checkpoint_{accuracy:.2f}.keras'
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='loss',
    verbose=1,
    save_best_only=False,
    save_freq=save_freq,#int(50000/sequence_length),#int(832000000 / (model_scale * num_transformer_blocks * batch_size * vocab_size * sequence_length)),  # настраиваемое значение
    #save_format='tf'  # Использование формата TensorFlow SavedModel
)
#print(f'save_freq = {int(832000000 / (model_scale * num_transformer_blocks * batch_size * vocab_size * sequence_length))}')
print(f'save_freq = {int(50000/sequence_length)}')

# Настройка callback для разогрева learning rate
lr_callback = LearningRateScheduler(lr_scheduler)

# Добавление callback в список callbacks
if use_lr_scheduler:
    callbacks = [checkpoint_callback, lr_callback]
else:
    callbacks = [checkpoint_callback, PrintLearningRate()]

print("Обучение модели")

start_time = time.time()

# Обучение модели с отображением прогресса
print(f"model_scale = {model_scale}, num_transformer_blocks = {num_transformer_blocks}")
print(f"sequence = {sequence_length}, batch = {batch_size}")
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")

    model.fit(
        distributed_dataset,
        epochs=epochs_infit,
        steps_per_epoch=steps_per_epoch,
        callbacks=callbacks
    )

    elapsed_time = time.time() - start_time
    minutes = int(elapsed_time // 60)
    seconds = int(elapsed_time % 60)
    print(f"Total elapsed time: {minutes}:{seconds}")

In [ ]:
# ДВА МЕТОДА БАТЧИНГА, RAM И ДИСК

# ДИСК
# Функция для генерации батчей из файла
def generate_batch(file_path, batch_size, sequence_length):
    indices = np.random.randint(0, token_count - sequence_length - 1, batch_size)
    X = np.zeros((batch_size, sequence_length), dtype=np.int64)
    Y = np.zeros((batch_size, sequence_length), dtype=np.int64)

    with open(file_path, 'rb') as f:
            for i, idx in enumerate(indices):
                # Чтение X
                f.seek(idx * 8)  # np.int64 занимает 8 байт
                x_seq = np.fromfile(f, dtype=np.int64, count=sequence_length)

                # Чтение Y
                f.seek((idx + 1) * 8)  # смещение на 1 элемент
                y_seq = np.fromfile(f, dtype=np.int64, count=sequence_length)

                if np.any(x_seq >= vocab_size) or np.any(y_seq >= vocab_size):
                    print(f"Found token with index >= {vocab_size}")
                    continue

                X[i] = x_seq
                Y[i] = y_seq

    return X, Y


# RAM
# Функция для генерации батчей из массива токенов
def generate_batch(tokens, batch_size, sequence_length):
    indices = np.random.randint(0, len(tokens) - sequence_length - 1, batch_size)
    X = np.zeros((batch_size, sequence_length), dtype=np.int64)
    Y = np.zeros((batch_size, sequence_length), dtype=np.int64)

    for i, idx in enumerate(indices):
        X[i] = tokens[idx:idx + sequence_length]
        Y[i] = tokens[idx + 1:idx + 1 + sequence_length]

    return X, Y


# НЕ НАШЕЛ ПОЛЕЗНЫМ
def batch_generator(file_path, batch_size, sequence_length, steps_per_epoch):
    for _ in range(steps_per_epoch):
        yield generate_batch(file_path, batch_size, sequence_length)


# внутри цикла эпох
#dataset = tf.data.Dataset.from_generator(
    #    lambda: batch_generator(dir_path + 'tokens.npy', batch_size, sequence_length, steps_per_epoch),
    #     lambda: generate_batch(dir_path + 'tokens.npy', batch_size, sequence_length),
    #    output_signature=(
    #        tf.TensorSpec(shape=(batch_size, sequence_length), dtype=tf.int64),
    #        tf.TensorSpec(shape=(batch_size, sequence_length), dtype=tf.int64),
    #    )
    #)

    # Применение параллельной обработки
    #dataset = dataset.map(lambda x, y: (x, y), num_parallel_calls=tf.data.experimental.AUTOTUNE)
    #dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)


# ДЛЯ TPU ИСПОЛЬЗУЙ TOKENS, не .npy
dataset = tf.data.Dataset.from_generator(# 'tokens.npy'
    lambda: iter([generate_batch(tokens, batch_size, sequence_length)]),
    output_signature=(
        tf.TensorSpec(shape=(batch_size, sequence_length), dtype=tf.int64),
        tf.TensorSpec(shape=(batch_size, sequence_length), dtype=tf.int64),
    )
)
dataset = dataset.repeat().prefetch(tf.data.experimental.AUTOTUNE)


#tf.keras.backend.clear_session() # не помогло сбросить TPU
try:
    tpu_address = os.environ.get('COLAB_TPU_ADDR')
    if tpu_address:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + tpu_address)
    else:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
except ValueError:
    tpu = None
if tpu:
    print("TPU = True")
else:
    print("TPU = False")
if tpu:
    try:
        tf.config.experimental_connect_to_cluster(tpu)
        if 'strategy' not in globals():
            tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.TPUStrategy(tpu)
        print('Running on TPU')
    except Exception as e:
        print(f"Failed to initialize TPU: {e}")
        strategy = tf.distribute.get_strategy()
else:
    # Если TPU недоступен, используем стратегию по умолчанию
    strategy = tf.distribute.get_strategy()
    print('Running on default strategy (CPU/GPU)')


if create_nn:
    print("Создание модели")
    class TransformerBlock(layers.Layer):
        def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
            super(TransformerBlock, self).__init__()
            self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
            self.ffn = tf.keras.Sequential(
                [layers.Dense(ff_dim, activation="relu"),
                 layers.Dense(embed_dim),]
            )
            self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
            self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
            self.dropout1 = layers.Dropout(rate)
            self.dropout2 = layers.Dropout(rate)

        def call(self, inputs, training):
            attn_output = self.att(inputs, inputs)
            attn_output = self.dropout1(attn_output, training=training)
            out1 = self.layernorm1(inputs + attn_output)
            ffn_output = self.ffn(out1)
            ffn_output = self.dropout2(ffn_output, training=training)
            return self.layernorm2(out1 + ffn_output)

    embed_dim = 16 * model_scale  # размерность вложения
    num_heads = 1 * model_scale  # количество голов в механизме внимания
    ff_dim = 16 * model_scale  # размерность полносвязного слоя
    with strategy.scope():
        inputs = tf.keras.Input(shape=(None,))
        embedding_layer = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        x = embedding_layer(inputs)
        for _ in range(num_transformer_blocks):
            transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
            x = transformer_block(x)
        outputs = layers.Dense(vocab_size, activation='softmax')(x)
        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        print("Компиляция модели")
        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
else:
    # Настройка модели внутри TPU стратегии
    with strategy.scope():
        print("Загрузка обученной модели")
        model = tf.keras.models.load_model(my_model_path)

        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])


! gcloud compute tpus locations list --project=project
! gcloud compute tpus stop --zone=ZONE


# Функция для поиска оптимального learning rate
def find_optimal_lr(model, dataset, steps, init_value=1e-7, final_value=10, beta=0.98):
    num = steps - 1
    mult = (final_value / init_value) ** (1/num)
    lr = init_value
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    avg_loss = 0.
    best_loss = 0.
    batch_num = 0
    losses = []
    log_lrs = []

    @tf.function
    def step_fn(inputs, targets):
        with tf.GradientTape() as tape:
            predictions = model(inputs, training=True)
            loss = model.compiled_loss(targets, predictions)
        return loss

    for step, (inputs, targets) in enumerate(dataset.take(steps)):
        print(f"\rFind_lr_step = {step+1}/{steps}", end='', flush=True)

        batch_num += 1
        loss = strategy.run(step_fn, args=(inputs, targets))
        loss = strategy.reduce(tf.distribute.ReduceOp.MEAN, loss, axis=None)

        avg_loss = beta * avg_loss + (1 - beta) * loss.numpy()
        smoothed_loss = avg_loss / (1 - beta**batch_num)

        # здесь 4 это threshold, ловит взрыв loss в х4
        if batch_num > 1 and smoothed_loss > 4 * best_loss:
            break

        if smoothed_loss < best_loss or batch_num == 1:
            best_loss = smoothed_loss

        losses.append(smoothed_loss)
        log_lrs.append(np.log10(lr))

        #grads = tape.gradient(loss, model.trainable_variables)
        #optimizer.apply_gradients(zip(grads, model.trainable_variables))

        lr *= mult
        optimizer.learning_rate.assign(lr)

    slopes = [(losses[i+1] - losses[i]) / (log_lrs[i+1] - log_lrs[i]) for i in range(len(losses)-1)]
    min_slope_index = np.argmin(slopes)
    optimal_lr = 10**log_lrs[min_slope_index]

    return optimal_lr


# Находим оптимальное значение learning rate
steps = 10  # Количество шагов для поиска оптимального learning rate
init_value = 1e-7
final_value = 1e-2
dataset = generate_dataset_ram(tokens, batch_size, sequence_length)
if find_lr:
    optimal_lr = find_optimal_lr(model, dataset, steps, init_value=init_value, final_value=final_value)
else:
    optimal_lr = initial_lr
print(f"\nOptimal Learning Rate: {optimal_lr:.4e}")

# Создаем оптимизатор с начальной learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=optimal_lr)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# БЕНЧМАРК

import timeit
import numpy as np

# Функция для вычисления чисел Фибоначчи
def fibonacci(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)

# Функция для выполнения матричных операций с NumPy
def matrix_operations(size):
    A = np.random.rand(size, size)
    B = np.random.rand(size, size)
    C = np.dot(A, B)
    return C

# Количество итераций для измерений
iterations = 10
fibonacci_time = 0
matrix_time = 0
for i in range(10):
    # Измерение времени выполнения вычислений чисел Фибоначчи
    fibo = 30
    fibonacci_time = (fibonacci_time*i + timeit.timeit("fibonacci(fibo)", globals=globals(), number=iterations))/(i+1)
    print(f"Время выполнения вычислений чисел Фибоначчи: {fibonacci_time / iterations:.6f} секунд на итерацию")

    # Измерение времени выполнения матричных операций
    matrix_size = 1600
    matrix_time = (matrix_time*i + timeit.timeit("matrix_operations(matrix_size)", globals=globals(), number=iterations))/(i+1)
    print(f"Время выполнения матричных операций: {matrix_time / iterations:.6f} секунд на итерацию\n")

# Создание файла с результатами
with open("benchmark_results.txt", "w") as f:
    f.write(f"Время выполнения вычислений чисел Фибоначчи: {fibonacci_time / iterations:.6f} секунд на итерацию\n")
    f.write(f"Время выполнения матричных операций: {matrix_time / iterations:.6f} секунд на итерацию\n")

In [ ]:
# СПЛИТ ДАТА ФАЙЛА

def split_file(input_file, output_file1, output_file2):
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    mid_index = len(lines) // 2

    with open(output_file1, 'w', encoding='utf-8') as f1:
        f1.writelines(lines[:mid_index])

    with open(output_file2, 'w', encoding='utf-8') as f2:
        f2.writelines(lines[mid_index:])

# Пример использования
input_file = '22.txt'
output_file1 = '221.txt'
output_file2 = '222.txt'

split_file(input_file, output_file1, output_file2)
print("Файл успешно разделен на два файла.")

In [ ]:
# ИНФЕРЕНС СЕТИ TOPPK

import tensorflow as tf
import numpy as np
from tokenizers import Tokenizer
import ipywidgets as widgets
from IPython.display import display
# Загружаем модель и токенизатор
dir_path = ''
tokenizer = Tokenizer.from_file(dir_path + my_tokenizer_path)
model_path = dir_path + my_model_path
model = tf.keras.models.load_model(model_path)

def log_softmax(logits):
    return logits - np.log(np.sum(np.exp(logits), axis=-1, keepdims=True))

# Функция для выборки следующего токена
def sample(predictions, temperature=1.0, top_k=0, top_p=0.0):
    predictions = np.asarray(predictions).astype('float64')

    # Применение temperature scaling
    predictions = predictions / temperature

    # Применение log_softmax
    logprobs = log_softmax(predictions)

    # Применение top-k sampling
    if top_k > 0:
        top_k_indices = np.argsort(logprobs)[-top_k:]
        top_k_logprobs = logprobs[top_k_indices]
        top_k_probs = np.exp(top_k_logprobs)
        top_k_probs /= np.sum(top_k_probs)
        sorted_indices = top_k_indices
        sorted_probs = top_k_probs
    else:
        sorted_indices = np.argsort(logprobs)[::-1]
        sorted_probs = np.exp(logprobs[sorted_indices])
        sorted_probs /= np.sum(sorted_probs)

    # Применение top-p sampling
    if top_p > 0.0:
        cumulative_probs = np.cumsum(sorted_probs)
        cutoff = np.argmax(cumulative_probs > top_p)
        filtered_indices = sorted_indices[:cutoff + 1]
        filtered_probs = sorted_probs[:cutoff + 1]
    else:
        filtered_indices = sorted_indices
        filtered_probs = sorted_probs

    # Выбор следующего токена
    if False:
        filtered_probs /= np.sum(filtered_probs)  # Нормализация
        chosen_index = np.random.choice(filtered_indices, p=filtered_probs)
        return chosen_index
    else:
        filtered_preds = np.zeros(len(predictions))
        filtered_preds[filtered_indices] = filtered_probs
        filtered_preds /= np.sum(filtered_preds)
        probas = np.random.multinomial(1, filtered_preds, 1)
        return np.argmax(probas)

# Функция для генерации текста
def generate_text(model, tokenizer, input_text, max_length=100, temperature=1.0, top_k=0, top_p=0.0):
    tokens = tokenizer.encode(input_text).ids
    for _ in range(max_length):
        token_array = np.array(tokens)[None, :]  # Add batch dimension
        predictions = model.predict(token_array, verbose=0)
        next_token_logits = predictions[0, -1, :]

        next_token = sample(next_token_logits, temperature=temperature, top_k=top_k, top_p=top_p)
        tokens.append(next_token)

        # Отображаем ответ с добавлением новых токенов в той же строке
        #print(f"\rText: {tokenizer.decode(tokens)}", end='', flush=True)

        if next_token == tokenizer.token_to_id("[END]"):
            break

    return tokenizer.decode(tokens)

# Определяем параметры генерации
temperature = 1
top_k = 50
top_p = 0.69
max_length = 5

# Взаимодействие с моделью
while False:
    #input_text = input("\nYou: ")
    input_text = my_input_text
    if input_text.lower() in ["exit", "quit"]:
        break
    response = generate_text(model, tokenizer, input_text, max_length=max_length, temperature=temperature, tail_free_threshold=tail_free_threshold)
    print(f"AI:{response}")
#for i in range(5,15+1, 2):
#    for j in range(5, 15+1, 2):
#        print(f"\ntemp={i*0.1} tfs={j*0.1}")
#        print(generate_text(model, tokenizer, "такой кайф", max_length=max_length, temperature=i * 0.1, tail_free_threshold=j * 0.1))

# Создаем виджеты
text_input = widgets.Text(
    value='',
    placeholder='You',
    description='You:',
    disabled=False
)

output_area = widgets.Output()

def on_button_click(b):
    with output_area:
        output_area.clear_output()
        prompt = text_input.value
        response = generate_text(model, tokenizer, prompt, max_length=max_length, temperature=temperature, top_k=top_k, top_p=top_p)
        print(f"\rAI: {response}", end='', flush=True)

button = widgets.Button(description="Сгенерировать текст")
button.on_click(on_button_click)

# Add widgets for temperature and top_p
temperature_slider = widgets.FloatSlider(
    value=temperature,
    min=0.1,
    max=2.0,
    step=0.1,
    description='Temp:',
    continuous_update=False
)

top_p_slider = widgets.FloatSlider(
    value=top_p,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Top_p:',
    continuous_update=False
)

def update_params(change):
    global temperature
    global top_p
    temperature = temperature_slider.value
    top_p = top_p_slider.value

temperature_slider.observe(update_params, names='value')
top_p_slider.observe(update_params, names='value')

# Отображаем виджеты
display(text_input)
display(button)
display(temperature_slider)
display(top_p_slider)
display(output_area)